# **Rendering tool for high quality molecular movies by POVARY**

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />
<p style="text-align: justify;font-size:15px">  
    This is an application to render high quality molecular structures by using the POVRAY
    as the rendering engine. One need upload XYZ trajectory files and obtain high quality
    movies (animinations). 
</p>

<p style="text-align: justify;font-size:15px">  
    One can visualize the molecular structures from the Nglview visualizer. One can pick 
    the orientation from the visualizer. User can choice the starting frame by moving the
    slider in the Nglview visualizer.
</p>

In [ ]:
from ase.build import molecule
from widget_periodictable import PTableWidget
import nglview as nv
from ipywidgets import HBox, VBox, Textarea, Button, Layout, ColorPicker
from ipywidgets import Checkbox, FileUpload, FloatSlider, Image, Dropdown, FloatProgress
import numpy as np
from vapory import *
from numpy.linalg import norm
from copy import deepcopy
from ase.io import read, write
import matplotlib
import io
import json
import base64
from IPython.display import Javascript
from moviepy.editor import VideoClip
from ase.io.trajectory import Trajectory

In [ ]:
style = {'description_width': 'initial'}

with open("colors.json", "r") as fs:
    tcolors = json.load(fs)
    
with open("radius.json", "r") as fx:
    radius = json.load(fx)
    
fs.close()
fx.close()

color_theme = Dropdown(
    options=['jmol', 'xcrysden'],
    value='jmol',
    description='Color theme: ',
    disabled=False,
    style = style
)

In [ ]:
view = nv.NGLWidget(gui=False)
view.camera='perspective'
view.background='white'

cp = ColorPicker(
    concise=False,
    description='Background color',
    value='white',
    disabled=False,
    style = style
)

def background_color_change(b):
    view.background = cp.value

cp.observe(background_color_change, names="value")

pb = FloatProgress(
    value=0.0,
    min=0,
    max=1.0,
    step=0.01,
    description='Rendering: ',
    bar_style='warning',
    orientation='horizontal'
)


In [ ]:
def _on_file_upload(change=None):
    global cc, dd
    """When file upload button is pressed."""
    for fname, item in change['new'].items():
        frmt = fname.split('.')[-1]
        if frmt == 'xyz':
            cc = read(io.StringIO(item['content'].decode()), format='xyz', index=':')
            for i in range(len(cc)):
                cc[i].set_cell([[15, 0, 0], [0, 15, 0], [0, 0, 15]])
                cc[i].center()
                cc[i].pbc=True
            write("1.traj", cc)
            dd = Trajectory("1.traj")
        break
            

fupload = FileUpload(
    accept='.xyz',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

fupload.observe(_on_file_upload, names='value')

bond_factor = FloatSlider(value = 1.2, min = 0.5, max = 2.0, description="Factor k: ", style = style)

In [ ]:
PTable = PTableWidget(states=2, selected_colors = ['red','green'], selected_elements = {'C':0}, 
                      border_color='black', unselected_color = 'pink', width='15px')

In [ ]:
def _prepare_payload(file_format=None):
    """Prepare binary information."""
    with open('movie.mp4', 'rb') as raw:
        return base64.b64encode(raw.read()).decode()
    
    
def _download(payload, filename):
    """Download payload as a file named as filename."""
    javas = Javascript("""
        var link = document.createElement('a');
        link.href = "data:;base64,{payload}"
        link.download = "{filename}"
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
        """.format(payload=payload, filename=filename))
    display(javas)

In [ ]:
nframe = 0;
nstart = 0;

def on_fileupload_click(b):
    global view
    
    for comp_id in view._ngl_component_ids:
        view.remove_component(comp_id)

    view.add_trajectory(nv.ASETrajectory(dd), default_representation=True)
    view.add_ball_and_stick(aspectRatio=4)
    view.add_unitcell()
    bcell.value = True
    view.control.zoom(0.2)
    PTable.selected_elements = {key: 0 for key in list(dict.fromkeys(cc[0].get_chemical_symbols()))}
    fupload.value.clear()
    fupload._counter = 0


def scene(t):
    colors = tcolors[color_theme.value]
    A = Bmatrix
    A = np.array(A)
    A=A.reshape(4,4)
    A=np.transpose(A)

    zfactor = norm(A[0, 0:3])
    A[0:3, 0:3] = A[0:3, 0:3]/zfactor
    
    bb = deepcopy(cc[nframe]);
    bb.set_cell([[15, 0, 0], [0, 15, 0], [0, 0, 15]])
    bb.center()
    bb.pbc=True;

    for i in bb:
        a = np.array([i.x, i.y, i.z])
        a = a + A[0:3, 3];
        w = A[0:3, 0:3].dot(a)
        i.x = -w[0]
        i.y = w[1]
        i.z = w[2]
        
    vertices = [];
    
    vx = np.array(bb.get_cell()[0]);
    vy = np.array(bb.get_cell()[1]);
    vz = np.array(bb.get_cell()[2]);

    vertices.append(np.array([0, 0, 0]));
    vertices.append(vx);
    vertices.append(vy);
    vertices.append(vz);
    
    vertices.append(vx+vy);
    vertices.append(vx+vz);
    vertices.append(vy+vz);
    vertices.append(vx+vy+vz);


    for n, i in enumerate(vertices):
        a = i + A[0:3, 3];
        w = A[0:3, 0:3].dot(a)
        vertices[n] = np.array([-w[0], w[1], w[2]])
    

    camera = Camera('location', [0, 0, -zfactor/1.5], 'look_at', [0.0, 0.0, 0.0])

    light1 = LightSource([0, 0, -100.0], 'color',  [1.5, 1.5, 1.5])
    light2 = LightSource([-100.0, -100.0, -60], 'color',  [1.5, 1.5, 1.5])
    light3 = LightSource([0, -60.0, 0], 'color',  [1, 1, 1])

    #light = LightSource(-A[2][0:3], [1.3, 1.3, 1.3])
    wall = Plane([0, 0, 100], 20, Texture(Pigment('color', [1, 1, 1])))

    spheres = [];

    for i in bb:
        sphere = Sphere( [i.x, i.y, i.z], radius[i.symbol], 
                        Texture(Pigment( 'color', np.array(colors[i.symbol]))), 
                            Finish('phong', 0.9,'reflection', 0.05))
        spheres.append(sphere)


    bonds = [];
    for x, i in enumerate(bb):
        for j in bb[x+1:]:
            v1 = np.array([i.x, i.y, i.z])
            v2 = np.array([j.x, j.y, j.z])

            if i.symbol == 'H' and j.symbol == 'H':
                continue;
                
            if norm(v1-v2) < bond_factor.value*(radius[i.symbol] + radius[j.symbol]):
                midi = v1 + (v2-v1)*radius[i.symbol]/(radius[i.symbol] + radius[j.symbol]);
                bond = Cylinder(v1, midi, 0.2, Pigment('color', np.array(colors[i.symbol])),
                                Finish('phong', 0.8,'reflection', 0.05))
                bonds.append(bond)
                bond = Cylinder(v2, midi, 0.2, Pigment('color', np.array(colors[j.symbol])),
                                Finish('phong', 0.8,'reflection', 0.05))
                bonds.append(bond)
                
    edges = [];
    for x, i in enumerate(vertices):
        for j in vertices:
            if abs(norm(i-j)-norm(vx)) < 0.001 and abs(norm(i-j)-norm(vy)) < 0.001 and abs(norm(i-j)-norm(vz)) < 0.001:
                edge = Cylinder(i, j, 0.06, Texture(Pigment( 'color', [212/255.0,175/255.0,55/255.0])), 
                            Finish('phong', 0.9,'reflection', 0.01))
                edges.append(edge)

    objects = [light1] + spheres + bonds + [Background( "color", np.array(matplotlib.colors.to_rgb(view.background)))]
    
    if bcell.value:
        objects += edges
        
    return Scene( camera, objects= objects)


def make_frame(t):
    global nframe 
    pb.value = (nframe - nstart + 1.0)/float(len(cc) - nstart)
    nframe+=1
    return scene(t).render(width = 2000, height= 2000, antialiasing=0.001)    


def make_movie(c):
    global nframe, nstart, Bmatrix
    
    fig_download.disabled = True
    ba.disabled = True
    
    nframe = view.frame
    nstart = view.frame
    Bmatrix = view._camera_orientation
    VideoClip(make_frame, duration=int(len(cc)/10)).write_videofile("movie.mp4",fps=10)
    
    ba.disabled = False
    fig_download.disabled = False
    
    
ba = Button(description = 'Render')
bu = Button(description = 'Upload File')
ba.on_click(make_movie)

bcell = Checkbox(
    value=True,
    description='Show cellbox',
    disabled=False,
    indent=False
)

        
def on_figure_download(b):
    _download(payload=_prepare_payload(), filename="movie.mp4")

    
fig_download = Button(description = 'Download Movie', disabled=True)

bu.on_click(on_fileupload_click)
fig_download.on_click(on_figure_download)

element_radius = FloatSlider(value = 0.1, min = 0.1, max = 2.0, 
                             description="Radius: ",
                             style = style)

def checkbox_change(b):
    global view
    if bcell.value:
        view.add_unitcell()
    else:
        view.remove_unitcell()

bcell.observe(checkbox_change, names='value')

selected = 'DD'
def change_radius(b):
    global selected
    elements = PTable.selected_elements;
    for x, y in elements.items():
        if y == 1:
            element_radius.value = radius[x]
            selected = x

def on_element_change(b):
    global radius
    
    if selected in PTable.selected_elements:
        if PTable.selected_elements[selected] == 1:
            radius[selected] = element_radius.value 
    
PTable.observe(change_radius, names = 'selected_elements')
element_radius.observe(on_element_change, names = 'value')

**Upload a XYZ trajectory file**

In [ ]:
display(HBox([fupload, bu]))

**Choice element color theme and background color**

In [ ]:
display(color_theme, cp)

**Choice whether to plot cellbox**

In [ ]:
display(bcell)

**Determinate the cutoff for bonds**

<p style="text-align: justify;font-size:15px"> 
The factor k determinates whether to form a bond between atom A and atom B. 
It will draw bonds when the distance between atom A and B is smaller than $k*(radius(A)+radius(B))$.
<p>

In [ ]:
display(bond_factor)

In [ ]:
display(view, pb, HBox([ba, fig_download]))

**Modify the radius of the atoms**

<p style="text-align: justify;font-size:15px"> 
    By clicking the elements in the peridic table, one can toggle the elements into green color.
    When the element is in green color, one can see the default radius in the silder. One can 
    also modify the radius by tuning the slider.
</p>

In [ ]:
display(element_radius, PTable)